In [ ]:
import torch
import time
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from data.swarmset import ContinuingDataset, SwarmDataset
from src.networks.embedding import NoveltyEmbedding
from src.networks.archive import DataAggregationArchive
from src.networks.ensemble import Ensemble
import numpy as np
from scipy import ndimage
import random
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

def CSVLineToVec(line):
    line_list = line.strip().replace("\n", "").split(",")
    float_list = []
    for i in line_list:
        float_list.append(float(i))
    float_list = np.array(float_list)
    return float_list

In [ ]:
# For single Sensor Baseline Model
TRUTH_FILE = "validation-data-baseline.txt"
OUT = "../data/oracle"
classes = [-1 for i in range(1000)]
controllers = []
sampled_dataset = SwarmDataset("../data/validation-easy-model", rank=0)

with open(os.path.join(OUT, TRUTH_FILE), "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        triplet = CSVLineToVec(line)
        classes[int(triplet[0])] = int(triplet[1])
        controllers.append(sampled_dataset[i][1])

for i in range(len(controllers)):
    print(f"{classes[i]}, {controllers[i][0]}, {controllers[i][1]}, {controllers[i][2]}, {controllers[i][3]}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ensemble = Ensemble(size=3, output_size=5, lr=15e-4, learning_decay=0.7, decay_step=1, threshold=9.0, weight_decay=1e-4, new_model=True, init="Random")
# ensemble.load_ensemble("../checkpoints/ensembles/01-20-23-baseline", full=True)
# ensemble.load_ensemble("../checkpoints/ensembles/01-20-23-filtered-C", full=True)
# ensemble.load_ensemble("../checkpoints/ensembles/01-23-23-two-s-pre-final", full=True)
ensemble.load_ensemble("../checkpoints/ensembles/01-23-23-baseline-hil-A", full=True)
ensemble.eval_mode()

# Source the Original Dataset
sampled_dataset = SwarmDataset("../data/validation-easy-model", rank=0)
# sampled_dataset = SwarmDataset("../data/validation-two-sensor-model", rank=0)
# sampled_dataset = SwarmDataset("../data/filtered-full", rank=0)

a = []
for i in range(len(ensemble.ensemble)):
    embedded_positions = []
    for j, c in enumerate(classes):
        image, _ = sampled_dataset[j][0], sampled_dataset[j][1][0]
        image = np.expand_dims(image, axis=0)
        embed = ensemble.ensemble[i].forward(torch.tensor(image, device=device, dtype=torch.float))
        embed = embed.detach().cpu().squeeze(dim=0).numpy()
        embedded_positions.append(embed)

    # Evaluate Accuracy
    MAX_SEARCH = 30000
    correct, total = 0, 0
    for x, _classX in validation_set:
        for y, _classY in validation_set:
            if x == y:
                continue
            for z, _classZ in validation_set:
                if x == z or y == z:
                    continue
                # If _classX and _classY are both random, ignore.
                if _classX == 0 and _classY == 0:
                    continue
                if _classZ != _classX and _classX == _classY:
                    positive_dist = np.linalg.norm(embedded_positions[x] - embedded_positions[y])
                    negative_dist = np.linalg.norm(embedded_positions[x] - embedded_positions[z])
                    if positive_dist < negative_dist:
                        correct += 1
                    total += 1
                if total > 30000:
                    break
            if total > 30000:
                break
        if total > 30000:
            break

    acc = correct * 100 / total
    a.append(acc)
    print(f"Ensemble {i} ~ Accuracy: {acc}")

print(f"Average: {sum(a) / 3}")